# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
df = pd.read_csv("output/city_data.csv")


In [17]:
df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Cape Town,-33.93,18.42,14.00,76,75,8.20,ZA,1598623732
1,1,Beipiao,41.79,120.78,16.84,88,23,3.37,CN,1598623853
2,2,Fukuechō,32.69,128.84,27.84,85,1,6.17,JP,1598623853
3,3,Tiksi,71.69,128.87,6.16,78,48,2.45,RU,1598623854
4,4,São Filipe,14.90,-24.50,26.11,82,91,2.55,CV,1598623854


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [19]:
#Use the Lat and Lng as locations and Humidity as the weight.
locations = df[["Lat", "Lng"]]
weight = df["Humidity"]

In [20]:
#add a map and add heatmap layer to the map
city_data_fig = gmaps.figure()


In [21]:
#heat layer
city_heat_layer = gmaps.heatmap_layer(locations, weights = weight, dissipating = False, 
                                     max_intensity = 100, point_radius = 1)

In [22]:
city_data_fig.add_layer(city_heat_layer)

In [23]:
city_data_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [32]:
#ideal vacation city, max temp of 26, min temp of 19
df = df.loc[(df["Max Temp"] >= 19) & (df["Max Temp"] <=26)]

In [44]:
#humidity between 30 to 50 percent
df = df.loc[(df["Humidity"] >= 30) & (df["Humidity"] <=50)]

In [51]:
#windspeed between 2 to 5mph
df = df.loc[(df["Wind Speed"] >= 0) & (df["Wind Speed"] <=5)]

In [58]:
#cloudiness under 10
df = df.loc[df["Cloudiness"] <= 35]

In [59]:
#check number of vacation spots passing all above filters
df.count()

Unnamed: 0    4
City          4
Lat           4
Lng           4
Max Temp      4
Humidity      4
Cloudiness    4
Wind Speed    4
Country       4
Date          4
dtype: int64

In [61]:
#check dataframe
df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
233,233,Salta,-24.79,-65.41,19.00,45,0,3.10,AR,1598623996
474,474,Kushmurun,52.45,64.63,25.61,47,6,3.33,KZ,1598624137
511,511,Atasū,48.68,71.64,21.13,39,0,3.40,KZ,1598624159
515,515,Makungu,-8.73,35.28,24.68,39,0,4.24,TZ,1598624162


In [63]:
#drop Unnamed column

df = df.drop(["Unnamed: 0"], axis = 1)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [64]:
#Store into variable named hotel_df
hotel_df = df.copy()


In [67]:
#reset df index to loop through the df usnig index for google API calls
hotel_df.reset_index()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,233,Salta,-24.79,-65.41,19.00,45,0,3.10,AR,1598623996
1,474,Kushmurun,52.45,64.63,25.61,47,6,3.33,KZ,1598624137
2,511,Atasū,48.68,71.64,21.13,39,0,3.40,KZ,1598624159
3,515,Makungu,-8.73,35.28,24.68,39,0,4.24,TZ,1598624162


In [65]:
#Set parameters to search for hotels with 5000 meters.
search_radius = 5000
search_type = "lodging"

In [68]:
#create empty list to store hotel info
hotel = []

In [71]:
#run a loop to find the nearest hotel based on google maps API
for index, row in hotel_df.iterrows():
    search_coordinates = f"{row['Lat']},{row['Lng']}"
    
    #create a dictionary to hold parameters
    parameters = {
        "location": search_coordinates,
        "radius": search_radius,
        'type': search_type,
        "key":g_key
    }
    
    #set base_url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    request = requests.get(base_url, params = parameters)
    response = request.json()
    
    #if hotel info is retireved add it to hotel list else move to next
    try:
        hotel.append(response["results"][0]["name"])
    except:
        hotel.append("No hotels found within 5km radius")

In [73]:
#check hotels
hotel

['Hotel Alejandro I', 'Gostinitsa Pri Depo', 'Dostyk', 'Complex']

In [74]:
#add hotel to dataframe
hotel_df["Hotels"] = hotel

In [75]:
#verify new column is present in dataframe
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotels
233,Salta,-24.79,-65.41,19.00,45,0,3.10,AR,1598623996,Hotel Alejandro I
474,Kushmurun,52.45,64.63,25.61,47,6,3.33,KZ,1598624137,Gostinitsa Pri Depo
511,Atasū,48.68,71.64,21.13,39,0,3.40,KZ,1598624159,Dostyk
515,Makungu,-8.73,35.28,24.68,39,0,4.24,TZ,1598624162,Complex


In [82]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotels}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_1 = hotel_df[["Lat", "Lng"]]

In [85]:
# Add marker layer ontop of heat map
figure = gmaps.figure()
heat_map = gmaps.heatmap_layer(locations, weights = weight,
                                dissipating = False, max_intensity = 100,
                                point_radius = 1)

marker_layer = gmaps.marker_layer(locations_1)
fig.add_layer(heat_map)
fig.add_layer(marker_layer)

# Display figure
fig

ValueError: weights must be of the same length as locations or None